In [33]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np
from sklearn.ensemble import RandomForestClassifier

from hyperopt import hp
from hyperopt import fmin, tpe, space_eval
from hyperopt.pyll.base import scope


import mag
from mag.experiment import Experiment
from mag import summarize


from speech2phone.preprocessing import get_TIMIT, get_phones
from speech2phone.preprocessing.filters import mel

np.set_printoptions(precision=3)

# Random Forest with `mag`

In [11]:
X_toy, y_toy = get_TIMIT(dataset='toy', preprocessor=mel, TIMIT_root='../TIMIT/TIMIT')

Loading toy/mel set from cache... done.


In [26]:
def random_forest_acc(n_estimators, max_depth):
    rf = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=42)
    rf.fit(X_toy, y_toy)
    acc = rf.score(X_toy, y_toy)
    return acc



experiments_dir = '../experiments/results/random_forest_v1'
config = {
    'n_estimators': 11,
    'max_depth': 10
}
with Experiment(config=config, experiments_dir=experiments_dir) as experiment:
    config = experiment.config
    acc = random_forest_acc(config.n_estimators, config.max_depth)
    
    experiment.register_result('accuracy', acc)

# Random Forest with `hyperopt`

In [41]:
def acc_wrapper(args):
    acc = random_forest_acc(args['n_estimators'], args['max_depth'])
    print("{}: {:.3f}".format(args, acc))
    return acc

space = {
    'n_estimators': scope.int(hp.uniform('n_estimators', 1, 100)),
    'max_depth': scope.int(hp.uniform('max_depth', 1, 30))

}
best = fmin(acc_wrapper, space, algo=tpe.suggest, max_evals=5)
print("")
print("Best Raw:", best)
print("Best Readable:", space_eval(space, best))

{'max_depth': 1, 'n_estimators': 36}: 0.133
{'max_depth': 29, 'n_estimators': 62}: 1.000
{'max_depth': 18, 'n_estimators': 92}: 1.000
{'max_depth': 29, 'n_estimators': 38}: 1.000
{'max_depth': 9, 'n_estimators': 21}: 0.647

Best Raw: {'max_depth': 1.2407552544797074, 'n_estimators': 36.46446756647321}
Best Readable: {'max_depth': 1, 'n_estimators': 36}


# Random Forest with `mag` and `hyperopt`

In [42]:
def acc_wrapper(args):
    experiments_dir = '../experiments/results/random_forest_v2'
    with Experiment(config=args, experiments_dir=experiments_dir) as experiment:
        config = experiment.config
        acc = random_forest_acc(config.n_estimators, config.max_depth)
        experiment.register_result('accuracy', acc)
    return acc

space = {
    'n_estimators': scope.int(hp.uniform('n_estimators', 1, 100)),
    'max_depth': scope.int(hp.uniform('max_depth', 1, 30))

}
best = fmin(acc_wrapper, space, algo=tpe.suggest, max_evals=5)
print("")
print("Best Raw:", best)
print("Best Readable:", space_eval(space, best))


Best Raw: {'max_depth': 1.3105802146129977, 'n_estimators': 67.18887296488809}
Best Readable: {'max_depth': 1, 'n_estimators': 67}
